In [257]:
#31
import pandas as pd
import glob
import os

m_store=pd.read_csv('m_store.csv')
m_area=pd.read_csv('m_area.csv')

current_dir=os.getcwd()
tbl_order_file=os.path.join(current_dir,'tbl_order_*.csv')
tbl_order_files=glob.glob(tbl_order_file)

order_all=pd.DataFrame()
for file in tbl_order_files:
    order_tmp=pd.read_csv(file)
    print(f'{file}:{len(order_tmp)}')
    order_all=pd.concat([order_all,order_tmp],ignore_index=True)

order_all=order_all.loc[order_all['order_id']!=999]

order_all=pd.merge(order_all,m_store,on='store_id',how='left')
order_all=pd.merge(order_all,m_area,on='area_cd',how='left')

order_all.loc[order_all['takeout_flag']==0,'takeout_name']='デリバリー'
order_all.loc[order_all['takeout_flag']==1,'takeout_name']='お持ち帰り'

order_all.loc[order_all['status']==0,'status_name']='受付'
order_all.loc[order_all['status']==1,'status_name']='お支払済'
order_all.loc[order_all['status']==2,'status_name']='お渡し済'
order_all.loc[order_all['status']==9,'status_name']='キャンセル'

order_all.loc[:,'order_date']=pd.to_datetime(order_all['order_accept_date']).dt.date
order_all.head()

c:\Users\yoshikoshi\python_y\tbl_order_202004.csv:233260
c:\Users\yoshikoshi\python_y\tbl_order_202005.csv:241139
c:\Users\yoshikoshi\python_y\tbl_order_202006.csv:233301


,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name,order_date
0,79339111,49,C26387220,50,67393872,2020/4/1 11:00,2020/4/1 11:18,1,4144,1,浅草店,TK,東京,東京,お持ち帰り,お支払済,2020-04-01
1,18941733,85,C48773811,26,91834983,2020/4/1 11:00,2020/4/1 11:22,0,2877,2,目黒店,TK,東京,東京,デリバリー,お渡し済,2020-04-01
2,56217880,76,C66287421,36,64409634,2020/4/1 11:00,2020/4/1 11:15,0,2603,2,本郷店,TK,東京,東京,デリバリー,お渡し済,2020-04-01
3,28447783,190,C41156423,19,73032165,2020/4/1 11:00,2020/4/1 11:16,0,2732,2,栃木店,TO,北関東,栃木,デリバリー,お渡し済,2020-04-01
4,32576156,191,C54568117,71,23281182,2020/4/1 11:00,2020/4/1 11:53,0,2987,2,伊勢崎店,GU,北関東,群馬,デリバリー,お渡し済,2020-04-01


In [258]:
#31
import openpyxl

wb=openpyxl.Workbook()
ws=wb['Sheet']
ws.cell(1,1).value='書き込みテストです'
wb.save('test.xlsx')
wb.close()


In [259]:
#31
wb=openpyxl.load_workbook('test.xlsx',read_only=True)
ws=wb['Sheet']
print(ws.cell(1,1).value)
wb.close()

書き込みテストです


In [260]:
#31
store_id=1
store_df=order_all.loc[order_all['store_id']==store_id].copy()
store_name=store_df['store_name'].unique()[0]
store_sales_total=store_df.loc[store_df['status'].isin([1,2])]['total_amount'].sum()
store_sales_takeout=store_df.loc[store_df['status']==1]['total_amount'].sum()
store_sales_delivery=store_df.loc[store_df['status']==2]['total_amount'].sum()

print(f'売上額確認 {store_sales_total}={store_sales_takeout+store_sales_delivery}')
output_df=store_df[['order_accept_date','customer_id','total_amount','takeout_name','status_name']]
output_df.head()

売上額確認 9004535=9004535


,order_accept_date,customer_id,total_amount,takeout_name,status_name
115,2020/4/1 11:09,C25851661,2471,デリバリー,お渡し済
138,2020/4/1 11:11,C78632079,2112,デリバリー,キャンセル
332,2020/4/1 11:28,C44700154,2122,デリバリー,キャンセル
593,2020/4/1 11:49,C80269937,2615,お持ち帰り,お支払済
777,2020/4/1 12:05,C70409495,4692,デリバリー,キャンセル


In [261]:
#31
from openpyxl.utils.dataframe import dataframe_to_rows
store_title=f'{store_id}_{store_name}'

wb=openpyxl.Workbook()
ws=wb.active
ws.title=store_title
ws.cell(1,1).value=f'{store_title}売上データ'

rows=dataframe_to_rows(output_df,index=False,header=True)

row_start=3
col_start=2

for row_no,row in enumerate(rows,row_start):
    for col_no,value in enumerate(row,col_start):
        ws.cell(row_no,col_no).value=value
    
filename=f'{store_title}.xlsx'
wb.save(filename)
wb.close()    

In [262]:
#32
from openpyxl.styles import PatternFill,Border,Side,Font

openpyxl.load_workbook(filename)
ws=wb[store_title]

side=Side(style='thin',color='008080')
border=Border(top=side,bottom=side,left=side,right=side)

for row in ws:
    for cell in row:
        if ws[cell.coordinate].value:
            ws[cell.coordinate].border=border

In [263]:
#32
ws.cell(1,1).font=Font(bold=True,color='008080')
cellZ=ws.cell(3,2)
cell.fill=PatternFill(patternType='solid',fgColor='008080')
cell.value='注文受注日時'
cell.font=Font(bold=True,color='FFFFFF')

cell=ws.cell(3,3)
cell.fill=PatternFill(patternType='solid',fgColor='008080')
cell.value='顧客ID'
cell.font=Font(bold=True,color='FFFFFF')

cell=ws.cell(3,4)
cell.fill=PatternFill(patternType='solid',fgColor='008080')
cell.value='購入総額'
cell.font=Font(bold=True,color='FFFFFF')

cell=ws.cell(3,5)
cell.fill=PatternFill(patternType='solid',fgColor='008080')
cell.value='注文タイプ'
cell.font=Font(bold=True,color='FFFFFF')

cell=ws.cell(3,6)
cell.fill=PatternFill(patternType='solid',fgColor='008080')
cell.value='注文状態'
cell.font=Font(bold=True,color='FFFFFF')

ws.column_dimensions['A'].width=20
ws.column_dimensions['B'].width=20
ws.column_dimensions['C'].width=12
ws.column_dimensions['D'].width=12
ws.column_dimensions['F'].width=12
ws.column_dimensions['G'].width=12

wb.save(filename)
wb.close()

In [264]:
#33
def calc_delta(t):
    t1,t2=t
    dalta=t2-t1
    return dalta.total_seconds()/60

store_df.loc[:,'order_accept_datetime']=pd.to_datetime(store_df['order_accept_date'])
store_df.loc[:,'delivered_datetime']=pd.to_datetime(store_df['delivered_date'])
store_df.loc[:,'delta']=store_df[['order_accept_datetime','delivered_datetime']].apply(calc_delta,axis=1)

delivery_time=store_df.groupby(['store_id'])['delta'].describe()
delivery_time

,count,mean,std,min,25%,50%,75%,max
store_id,,,,,,,,
1,3553.0,34.477062,14.514403,10.0,22.0,34.0,47.0,59.0


In [265]:
#33
openpyxl.load_workbook(filename)
ws=wb[store_title]

cell=ws.cell(1,7)
cell.value=f'配達完了までの時間'
cell.font=Font(bold=True,color='008080')
rows=dataframe_to_rows(delivery_time,index=False,header=True)

row_start=3
col_start=8

for row_no,row in enumerate(rows,row_start):
    for col_no,value in enumerate(row,col_start):
        cell=ws.cell(row_no,col_no)
        cell.value=value
        cell.border=border
        if row_no==row_start:
            cell.fill=PatternFill(patternType='solid',fgColor='008080')
            cell.font=Font(bold=True,color='FFFFFF')

filename=f'{store_title}.xlsx'
wb.save(filename)
wb.close()

In [266]:
#34
openpyxl.load_workbook(filename)
ws=wb[store_title]
rows=dataframe_to_rows(output_df,index=False,header=True)

row_start=3
col_start=2

for row_no,row in enumerate(rows,row_start):
    if row_no==row_start:
        continue
    for col_no,value in enumerate(row,col_start):
        ws.cell(row_no,col_no).value=value
        if value=='キャンセル':
            ws.cell(row_no,col_no).font=Font(bold=True,color='FF0000')

filename=f'{store_title}.xlsx'
wb.save(filename)
wb.close()            

In [267]:
#35
openpyxl.load_workbook(filename)
ws=wb[store_title]

cell=ws.cell(7,7)
cell.value='集計'
cell.font=Font(bold=True,color='008080')

cell=ws.cell(8,8)
cell.value='データ総額'
cell.font=Font(bold=True,color='008080')

cell=ws.cell(8,10)
cell.value=f'=SUM(D4:D{ws.max_row})'

cell=ws.cell(9,8)
cell.value=' 内　決済完了額'
cell.font=Font(bold=True)

cell=ws.cell(9,10)
cell.value=f'=SUMIF(4:F{ws.max_row},"<>キャンセル",D4:D{ws.max_row})'

cell=ws.cell(10,8)
cell.value=' 内　キャンセル額'
cell.font=Font(bold=True)

cell=ws.cell(9,10)
cell.value=f'=SUMIF(4:F{ws.max_row},"=キャンセル",D4:D{ws.max_row})'

filename=f'{store_title}.xlsx'
wb.save(filename)
wb.close()


In [268]:
#36
from openpyxl.chart import Reference,BarChart,PieChart,LineChart,ScatterChart,Series

openpyxl.load_workbook(filename)
ws=wb[store_title]

cell=ws.cell(7,7)
cell.value=f'売上グラフ'
cell.font=Font(bold=True,color='008080')

refy=Reference(ws,min_col=4,min_row=4,max_col=4,max_row=23)
series=Series(refy,title='売上額')

chart=LineChart()
chart.title='折れ線グラフ'
chart.x_axis.title='件数'
chart.y_axis.title='売上額'
chart.height=10
chart.width=20
chart.series.append(series)
ws.add_chart(chart,'H12')

filename=f'{store_title}.xlsx'
wb.save(filename)
wb.close()


In [269]:
#37
cancel_df=pd.DataFrame()
cancel_cnt=order_all.loc[order_all['status']==9].groupby(['store_id'])['store_id'].count()
order_cnt=order_all.loc[order_all['status'].isin([1,2,9])].groupby(['store_id'])['store_id'].count()
cancel_rate=(cancel_cnt/order_cnt)*100
cancel_df['cancel_rate']=cancel_rate
cancel_df=pd.merge(cancel_df,m_store,on='store_id',how='left')
cancel_rank=cancel_df.sort_values('cancel_rate',ascending=True).reset_index()

def check_store_cancel_rank(trg_id):
    tmp=cancel_rank.loc[cancel_rank['store_id']==trg_id].index+1
    return tmp[0]

In [270]:
#37
def get_area_df(trg_id):
    area_df=pd.DataFrame()
    area_df=order_all.loc[order_all['area_cd']==store_df['area_cd'].unique()[0]]
    area_df=area_df.loc[area_df['status'].isin([1,2])]
    return area_df

def get_area_rank_df(trg_id):
    area_df=get_area_df(trg_id)
    area_rank=area_df.groupby(['store_id'])['total_amount'].sum().sort_values(ascending=False)
    area_rank=pd.merge(area_rank,m_store,on='store_id',how='left')
    return area_rank

def check_store_sales_rank(trg_id):
    area_rank=get_area_rank_df(trg_id)

    tmp=area_rank.loc[area_rank['store_id']==trg_id].index+1
    return tmp[0]

In [271]:
#37
def make_store_daily(trg_id):
    tmp_store_df=order_all.loc[(order_all['store_id']==trg_id) & (order_all['status'].isin([1,2]))]
    tmp=tmp_store_df[['order_accept_date','total_amount']].copy()
    tmp.loc[:,'order_accept_date']=pd.to_datetime(tmp['order_accept_date'])
    tmp.set_index('order_accept_date',inplace=True)
    tmp=tmp.resample('D').sum().reset_index()
    return tmp

In [272]:
#37
def get_area_delivery(trg_id):
    area_delivery=pd.DataFrame()
    area_df=get_area_df(trg_id)
    area_delivery=area_df.loc[area_df['status']==2].copy()

    area_delivery.loc[:,'order_accept_datetime']=pd.to_datetime(area_delivery['order_accept_date'])
    area_delivery.loc[:,'delivered_datetime']=pd.to_datetime(area_delivery['delivered_date'])
    area_delivery.loc[:,'delta']=area_delivery[['order_accept_datetime','delivered_datetime']].apply(calc_delta,axis=1)
    return area_delivery

def get_area_delivery_rank_df(trg_id):
    area_delivery=get_area_delivery(trg_id)
    area_delivery_rank=area_delivery.groupby(['store_id'])['delta'].mean().sort_values()
    area_delivery_rank=pd.merge(area_delivery_rank,m_store,on='store_id',how='left')
    return area_delivery_rank

def check_store_delivery_rank(trg_id):
    area_delivery_rank=get_area_delivery_rank_df(trg_id)
    tmp=area_delivery_rank.loc[area_delivery_rank['store_id']==trg_id].index+1
    return tmp[0]


In [273]:
#38
if os.path.exists('test.xlsx'):os.remove('test.xlsx')
if os.path.exists(filename):os.remove(filename)

In [274]:
#38
def data_sheet_output(trg_wb,sheetname,target_df,indexFlg):
    ws=trg_wb.create_sheet(title=sheetname)
    row=dataframe_to_rows(target_df,index=indexFlg,header=True)

    row_start=1
    col_start=1

    for row_no,row in enumerate(rows,row_start):
        for col_no,value in enumerate(row,col_start):
            ws.cell(row_no,col_no).value=value

    ws.sheet_state='hidden'        

In [275]:
#38
def make_data_sheet(trg_id,trg_st_df,targetfolder):
    target_daily=make_store_daily(trg_id)
    store_name=trg_st_df['store_name'].unique()[0]
    store_title=f'{trg_id}_{store_name}'
    wb=openpyxl.Workbook()

    data_sheet_output(wb,'Data_CancelRank',cancel_rank,False)
    data_sheet_output(wb,'Data_AreaRank',get_area_rank_df(trg_id),False)
    data_sheet_output(wb,'Data_DeliveryRank',get_area_delivery_rank_df,False)
    data_sheet_output(wb,'Data_Target_Daily',target_daily,False)

    filename=os.path.join(targetfolder,f'{store_title}.xlsx')
    wb.save(filename)
    wb.close()
    return filename

In [276]:
#38
filename_store=make_data_sheet(store_id,store_df,'')

C:\Users\yoshikoshi\AppData\Local\Temp\ipykernel_5392\3397485158.py:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  tmp.loc[:,'order_accept_date']=pd.to_datetime(tmp['order_accept_date'])


In [277]:
#39
def make_summary_sheet(trg_id,storename,trgfile):
    target_cancel_rank=check_store_cancel_rank(trg_id)
    target_sales_rank=check_store_sales_rank(trg_id)
    target_delivery_rank=check_store_delivery_rank(trg_id)

    wb=openpyxl.load_workbook(trgfile)
    ws=wb.active
    ws.title='サマリーレポート'

    cell=ws.cell(1,1)
    cell.value=f'{storename}　サマリーレポート　（４月～6月）'
    cell.font=Font(bold=True,color='008080',size=20)

    tmpWs=wb['Data_Target_Daily']
    cell=ws.cell(3,2)
    cell.value='店舗売上額'
    cell.font=Font(bold=True,color='008080',size=16)

    ws.merge_cells('E3:F3')

    cell=ws.cell(3,5)
    cell.value=f'=SUM({tmpWs.title}!B2:B{tmpWs.max_row})'
    cell.font=Font(bold=True,color='0080FF',size=16)
    cell.number_format='#,##0'

    cell=ws.cell(4,2)
    cell.value='店舗売上ランク'
    cell.font=Font(bold=True,color='008080',size=16)

    cell=ws.cell(4,5)
    cell.value=f'{len(m_store)}店舗中　{target_sales_rank}位'
    cell.font=Font(bold=True,color='0080FF',size=16)

    rety=Reference(tmpWs,min_col=2,min_row=2,max_col=2,max_row=tmpWs.max_row)
    series=Series(rety,title='売上額')

    chart=LineChart()
    chart.title='期間売上額（日毎）'
    chart.x_axis.title='件数'
    chart.y_axis.title='売上額'
    chart.height=10
    chart.width=15
    chart.series.append(series)

    ws.add_chart(chart,'B6')
    tmpWs=wb['Data_AreaRank']

    cell=ws.cell(4,10)
    cell.value='地域店舗売上情報'
    cell.font=Font(bold=True,color='008080',size=16)

    cell=ws.cell(5,11)
    cell.value='最高額'

    cell=ws.cell(5,12)
    cell.value=f'=MAX({tmpWs.title}!B2:B{tmpWs.max_row})'
    cell.number_format='#,##0'
    
    cell=ws.cell(6,11)
    cell.value='最低額'

    cell=ws.cell(6,12)
    cell.value=f'=MIN({tmpWs.title}!B2:B{tmpWs.max_row})'
    cell.number_format='#,##0'

    cell=ws.cell(7,11)
    cell.value='地域平均'

    cell=ws.cell(7,12)
    cell.value=f'=AVERAGE({tmpWs.title}!C2:C{tmpWs.max_row})'
    cell.number_format='#,##0'

    cell=ws.cell(11,10)
    cell.value='キャンセルランク'
    cell.font=Font(bold=True,color='008080',size=16)

    cell=ws.cell(12,11)
    cell.value=f'{len(m_store)}店舗中　{target_cancel_rank}位'
    cell.font=Font(bold=True,color='0080FF',size=16)

    tmpWs=wb['Data_CancelRank']

    cell=ws.cell(13,11)
    cell.value='地域平均'

    cell=ws.cell(13,12)
    cell.value=f'=AVERAGE({tmpWs.title}!C2:C{tmpWs.max_row})'
    cell.number_format='0.00'

    cell=ws.cell(15,10)
    cell.value='配達時間ランク'
    cell.font=Font(bold=True,color='008080',size=16)

    cell=ws.cell(16,11)
    cell.value=f'{len(m_store)}店舗中　{target_delivery_rank}位'
    cell.font=Font(bold=True,color='0080FF',size=16)

    tmpWs=wb['Data_DeliveryRank']

    cell=ws.cell(17,11)
    cell.value='地域平均'

    cell=ws.cell(17,12)
    cell.value=f'=AVERAGE({tmpWs.title}!B2:B{tmpWs.max_row})'
    cell.number_format='0.00'

    wb.save(trgfile)
    wb.close()


In [278]:
#39
make_summary_sheet(store_id,store_name,filename_store)

In [280]:
#40
os.makedirs('output',exist_ok=True)
for store in m_store['store_id'].tolist():
    if store != 999:
        store_df=order_all.loc[order_all['store_id']==store]
        store_name=m_store.loc[m_store['store_id']==store]['store_name']
        print(store_name)

        tmp_file_name=make_data_sheet(store,store_df,'output')
        make_summary_sheet(store,store_name.values[0],tmp_file_name)

print('出力完了しました')        

0    昭島店
Name: store_name, dtype: object


C:\Users\yoshikoshi\AppData\Local\Temp\ipykernel_5392\3397485158.py:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  tmp.loc[:,'order_accept_date']=pd.to_datetime(tmp['order_accept_date'])


1    あきる野店
Name: store_name, dtype: object


C:\Users\yoshikoshi\AppData\Local\Temp\ipykernel_5392\3397485158.py:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  tmp.loc[:,'order_accept_date']=pd.to_datetime(tmp['order_accept_date'])


2    足立店
Name: store_name, dtype: object


...

194    館林店
Name: store_name, dtype: object


C:\Users\yoshikoshi\AppData\Local\Temp\ipykernel_5392\3397485158.py:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  tmp.loc[:,'order_accept_date']=pd.to_datetime(tmp['order_accept_date'])


195    前橋店
Name: store_name, dtype: object


C:\Users\yoshikoshi\AppData\Local\Temp\ipykernel_5392\3397485158.py:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  tmp.loc[:,'order_accept_date']=pd.to_datetime(tmp['order_accept_date'])


出力完了しました
